In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [3]:
df_sample_submission = pd.read_csv('sample_submission.csv')


In [4]:
df_sample_submission = pd.read_csv('sample_submission.csv')
df_test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')

In [4]:
df_test.head(5)

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [5]:
df.head(5)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [6]:
df.shape,df_test.shape

((3116945, 22), (2077964, 21))

In [19]:
df_test.id

0          3116945
1          3116946
2          3116947
3          3116948
4          3116949
            ...   
2077959    5194904
2077960    5194905
2077961    5194906
2077962    5194907
2077963    5194908
Name: id, Length: 2077964, dtype: int64

In [20]:
df.iloc[1, :]

id                         1
class                      p
cap-diameter            4.51
cap-shape                  x
cap-surface                h
cap-color                  o
does-bruise-or-bleed       f
gill-attachment            a
gill-spacing               c
gill-color                 n
stem-height             4.79
stem-width              6.48
stem-root                NaN
stem-surface               y
stem-color                 o
veil-type                NaN
veil-color               NaN
has-ring                   t
ring-type                  z
spore-print-color        NaN
habitat                    d
season                     w
Name: 1, dtype: object

In [29]:
df['cap-color'].nunique()


78

In [28]:
exclude_cols = ["cap-diameter", "stem-height", "stem-width"]

unique_dict = {}
for col in df.columns:
    if col not in exclude_cols:
        unique_dict[col] = df[col].unique()

# تبدیل به DataFrame
unique_df = pd.DataFrame({k: pd.Series(v) for k, v in unique_dict.items()})

# ذخیره در Excel
unique_df.to_excel("unique_features.xlsx", index=False)

ValueError: This sheet is too large! Your sheet size is: 3116945, 19 Max sheet size is: 1048576, 16384

In [ ]:
df["cap-diameter"].value_counts

<bound method IndexOpsMixin.value_counts of 0           8.80
1           4.51
2           6.94
3           3.88
4           5.85
           ...  
3116940     9.29
3116941    10.88
3116942     7.82
3116943     9.45
3116944     3.20
Name: cap-diameter, Length: 3116945, dtype: float64>

In [25]:
for col in df.columns:
    # print(col, df[col].unique())
    print(col, df[col].value_counts)

    

id <bound method IndexOpsMixin.value_counts of 0                0
1                1
2                2
3                3
4                4
            ...   
3116940    3116940
3116941    3116941
3116942    3116942
3116943    3116943
3116944    3116944
Name: id, Length: 3116945, dtype: int64>
class <bound method IndexOpsMixin.value_counts of 0          e
1          p
2          e
3          e
4          e
          ..
3116940    e
3116941    e
3116942    p
3116943    e
3116944    p
Name: class, Length: 3116945, dtype: object>
cap-diameter <bound method IndexOpsMixin.value_counts of 0           8.80
1           4.51
2           6.94
3           3.88
4           5.85
           ...  
3116940     9.29
3116941    10.88
3116942     7.82
3116943     9.45
3116944     3.20
Name: cap-diameter, Length: 3116945, dtype: float64>
cap-shape <bound method IndexOpsMixin.value_counts of 0          f
1          x
2          f
3          f
4          x
          ..
3116940    f
3116941    s
3116942   

In [7]:
df_test.columns

Index(['id', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [8]:
df_test.isnull().sum() * 100 / df_test.shape[0]

id                       0.000000
cap-diameter             0.000337
cap-shape                0.001492
cap-surface             21.506821
cap-color                0.000626
does-bruise-or-bleed     0.000481
gill-attachment         16.834796
gill-spacing            40.404694
gill-color               0.002358
stem-height              0.000048
stem-width               0.000000
stem-root               88.452543
stem-surface            63.595327
stem-color               0.001011
veil-type               94.878689
veil-color              87.880445
has-ring                 0.000914
ring-type                4.148051
spore-print-color       91.417224
habitat                  0.001203
season                   0.000000
dtype: float64

In [9]:
df.isnull().sum() * 100 / df.shape[0]

id                       0.000000
class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.528227
cap-color                0.000385
does-bruise-or-bleed     0.000257
gill-attachment         16.809280
gill-spacing            40.373988
gill-color               0.001829
stem-height              0.000000
stem-width               0.000000
stem-root               88.452732
stem-surface            63.551362
stem-color               0.001219
veil-type               94.884350
veil-color              87.936970
has-ring                 0.000770
ring-type                4.134818
spore-print-color       91.425482
habitat                  0.001444
season                   0.000000
dtype: float64

In [ ]:
def class_encoder(df, name_column:str):
    new_column = []
    for value in df[name_column]:
        if value == 'p':
            new_column.append(1) # poisonous ast
        else:
            new_column.append(-1) #  poisonous nist
            
    df[name_column] = new_column


In [11]:
class_encoder(df,'class')

In [12]:
df.head(5)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,-1,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,1,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,-1,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,-1,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,-1,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [14]:
df['does-bruise-or-bleed'].value_counts(dropna=False)

does-bruise-or-bleed
f           2569743
t            547085
w                14
c                11
h                 9
NaN               8
x                 7
y                 7
a                 7
b                 7
s                 6
k                 6
d                 4
e                 4
p                 4
l                 4
z                 3
o                 3
g                 3
n                 2
i                 2
has-ring          1
r                 1
3.43              1
4.42              1
2.9               1
u                 1
Name: count, dtype: int64

In [ ]:
def no_unknown_yes_encoder(df, name_column: str):
    new_column = []
    for value in df[name_column]:
        if value == 'f':
            new_column.append(-1) # no
        elif value == 't':
            new_column.append(1) # yes
        else:
            new_column.append(0) # idk
            
    df[name_column] = new_column


In [16]:
no_unknown_yes_encoder(df,'does-bruise-or-bleed')
no_unknown_yes_encoder(df_test,'does-bruise-or-bleed')


In [17]:
df['season'].value_counts(dropna=False)

season
a    1543321
u    1153588
w     278189
s     141847
Name: count, dtype: int64

In [18]:
df['season'].value_counts(dropna=False)

season
a    1543321
u    1153588
w     278189
s     141847
Name: count, dtype: int64

In [ ]:
def season_encoder(df, name_column: str):
    new_season_cos = []
    new_season_sin = []
    for value in df[name_column]:
        if value == 's':  # autumn(paeez)
            new_season_cos.append(1)
            new_season_sin.append(0)
        elif value == 'w':  # Winter
            new_season_cos.append(0)
            new_season_sin.append(-1)
        elif value == 'a':  # Spring
            new_season_cos.append(-1)
            new_season_sin.append(0)
        elif value == 'u':  # Summer
            new_season_cos.append(0)
            new_season_sin.append(1)

    df['season_cos'] = new_season_cos
    df['season_sin'] = new_season_sin


In [20]:
season_encoder(df, 'season')
season_encoder(df_test, 'season')

In [21]:
df = df.drop(columns=['season'])
df_test = df_test.drop(columns=['season'])

In [22]:
df.columns

Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season_cos', 'season_sin'],
      dtype='object')

In [23]:
df.head(5)

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season_cos,season_sin
0,0,-1,8.80,f,s,u,-1,a,c,w,...,NaN,w,NaN,NaN,f,f,NaN,d,-1,0
1,1,1,4.51,x,h,o,-1,a,c,n,...,y,o,NaN,NaN,t,z,NaN,d,0,-1
2,2,-1,6.94,f,s,b,-1,x,c,w,...,s,n,NaN,NaN,f,f,NaN,l,0,-1
3,3,-1,3.88,f,y,g,-1,s,NaN,g,...,NaN,w,NaN,NaN,f,f,NaN,d,0,1
4,4,-1,5.85,x,l,w,-1,d,NaN,w,...,NaN,w,NaN,NaN,f,f,NaN,g,-1,0


In [24]:
multicategorical_columns = ['cap-shape', 'cap-surface', 'cap-color', 
                            'gill-attachment', 'gill-spacing', 'gill-color', 
                            'stem-root', 'stem-surface', 'stem-color',
                            'veil-type', 'veil-color', 'ring-type', 
                            'spore-print-color', 'habitat']

In [26]:
def abc_clearing(df, name_column: str):
    abc_categories = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h','i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'x', 'y', 'z'] 
    df[name_column] = df[name_column].mask(~df[name_column].isin(abc_categories))
    df[name_column] = df[name_column].fillna(0)
    # return df

In [36]:
abc_categories = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
                    'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 
                    's', 't', 'u', 'w', 'x', 'y', 'z', 0] 

freq_poisonous_full = {category: 0 for category in abc_categories}
# freq_edible_full = {category: 0 for category in abc_categories}
freq_poisonous = df[df['class'] == 1]['cap-shape'].value_counts(normalize=True)

type(freq_poisonous_full),
freq_poisonous.index

# freq_poisonous

Index(['x', 'f', 'b', 's', 'o', 'c', 'p', 'd', 't', 'z', 'e', 'y', 'n', 'r',
       'g', 'w', 'u', 'a', 'i', 'k', 'l', 'h', '2.85', '21.56', '3.6', '6 x',
       '4.22', 'ring-type', '0.82', '2.77', '12.62', '19.04', 'm', '7.21',
       '3.25', '3 x', '4.3', '7.41', '6.21', '3.52', '3.55', '3.04', '2.82'],
      dtype='object', name='cap-shape')

In [27]:

def abc_encoder(df, name_column):
    abc_categories = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
                      'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 
                      's', 't', 'u', 'w', 'x', 'y', 'z', 0] 
    
    freq_poisonous = df[df['class'] == 1][name_column].value_counts(normalize=True)
    freq_edible = df[df['class'] == -1][name_column].value_counts(normalize=True)

    freq_poisonous_full = {category: 0 for category in abc_categories}
    freq_edible_full = {category: 0 for category in abc_categories}

    for category in abc_categories:
        if category != 0:
            if category in freq_poisonous.index:
                freq_poisonous_full[category] += freq_poisonous[category]
            if category in freq_edible.index:
                freq_edible_full[category] += freq_edible[category]

    return freq_poisonous_full, freq_edible_full

In [38]:
def abc_column_processing(df, df_test, name_column: str):
    abc_clearing(df, name_column)
    abc_clearing(df_test, name_column)
    
    freq_p, freq_e = abc_encoder(df, name_column)

    df['freq-' + name_column + '-p'] = df[name_column].map(freq_p)
    df['freq-' + name_column + '-e'] = df[name_column].map(freq_e)

    df_test['freq-' + name_column + '-p'] = df_test[name_column].map(freq_p)
    df_test['freq-' + name_column + '-e'] = df_test[name_column].map(freq_e)

    df = df.drop(columns=[name_column], inplace=True)
    df_test = df_test.drop(columns=[name_column], inplace=True)

In [39]:
for column in multicategorical_columns:
    abc_column_processing(df, df_test, column)

In [40]:
df

,id,class,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,season_cos,season_sin,freq-cap-shape-p,...,freq-veil-type-p,freq-veil-type-e,freq-veil-color-p,freq-veil-color-e,freq-ring-type-p,freq-ring-type-e,freq-spore-print-color-p,freq-spore-print-color-e,freq-habitat-p,freq-habitat-e
0,0,-1,8.80,-1,4.51,15.39,f,-1,0,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
1,1,1,4.51,-1,4.79,6.48,t,0,-1,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.066461,0.000310,0.0,0.0,0.675260,0.726853
2,2,-1,6.94,-1,6.85,9.93,f,0,-1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.039805,0.073684
3,3,-1,3.88,-1,4.16,6.53,f,0,1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
4,4,-1,5.85,-1,3.37,8.36,f,-1,0,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.179948,0.104868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,-1,9.29,1,12.14,18.81,t,0,1,0.203000,...,0.061266,0.038887,0.082471,0.098066,0.015739,0.026103,0.0,0.0,0.675260,0.726853
3116941,3116941,-1,10.88,1,6.65,26.97,f,0,1,0.117645,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
3116942,3116942,1,7.82,-1,9.51,11.06,t,-1,0,0.429585,...,0.000000,0.000000,0.082471,0.098066,0.066461,0.000310,0.0,0.0,0.675260,0.726853
3116943,3116943,-1,9.45,1,9.13,17.77,t,0,1,0.028815,...,0.000000,0.000000,0.000000,0.000000,0.020403,0.023295,0.0,0.0,0.675260,0.726853


In [41]:
def no_yes_encoder(df, name_column: str):
    new_column = []
    for value in df[name_column]:
        if value == 'f':
            new_column.append(-1) # no
        elif value == 't':
            new_column.append(1) # yes
        else:
            new_column.append(0) # unknown
            
    df[name_column] = new_column


In [42]:
no_yes_encoder(df,'has-ring')
no_yes_encoder(df_test,'has-ring')

In [43]:
# df_test=df_test.dropna()

In [44]:
df_test.fillna(df.median(),inplace=True)

In [45]:
res2=df_test.isnull().sum() * 100 / df.shape[0]


In [46]:
df

,id,class,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,season_cos,season_sin,freq-cap-shape-p,...,freq-veil-type-p,freq-veil-type-e,freq-veil-color-p,freq-veil-color-e,freq-ring-type-p,freq-ring-type-e,freq-spore-print-color-p,freq-spore-print-color-e,freq-habitat-p,freq-habitat-e
0,0,-1,8.80,-1,4.51,15.39,-1,-1,0,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
1,1,1,4.51,-1,4.79,6.48,1,0,-1,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.066461,0.000310,0.0,0.0,0.675260,0.726853
2,2,-1,6.94,-1,6.85,9.93,-1,0,-1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.039805,0.073684
3,3,-1,3.88,-1,4.16,6.53,-1,0,1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
4,4,-1,5.85,-1,3.37,8.36,-1,-1,0,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.179948,0.104868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,-1,9.29,1,12.14,18.81,1,0,1,0.203000,...,0.061266,0.038887,0.082471,0.098066,0.015739,0.026103,0.0,0.0,0.675260,0.726853
3116941,3116941,-1,10.88,1,6.65,26.97,-1,0,1,0.117645,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
3116942,3116942,1,7.82,-1,9.51,11.06,1,-1,0,0.429585,...,0.000000,0.000000,0.082471,0.098066,0.066461,0.000310,0.0,0.0,0.675260,0.726853
3116943,3116943,-1,9.45,1,9.13,17.77,1,0,1,0.028815,...,0.000000,0.000000,0.000000,0.000000,0.020403,0.023295,0.0,0.0,0.675260,0.726853


In [47]:
res2[:]

id                          0.0
cap-diameter                0.0
does-bruise-or-bleed        0.0
stem-height                 0.0
stem-width                  0.0
has-ring                    0.0
season_cos                  0.0
season_sin                  0.0
freq-cap-shape-p            0.0
freq-cap-shape-e            0.0
freq-cap-surface-p          0.0
freq-cap-surface-e          0.0
freq-cap-color-p            0.0
freq-cap-color-e            0.0
freq-gill-attachment-p      0.0
freq-gill-attachment-e      0.0
freq-gill-spacing-p         0.0
freq-gill-spacing-e         0.0
freq-gill-color-p           0.0
freq-gill-color-e           0.0
freq-stem-root-p            0.0
freq-stem-root-e            0.0
freq-stem-surface-p         0.0
freq-stem-surface-e         0.0
freq-stem-color-p           0.0
freq-stem-color-e           0.0
freq-veil-type-p            0.0
freq-veil-type-e            0.0
freq-veil-color-p           0.0
freq-veil-color-e           0.0
freq-ring-type-p            0.0
freq-rin

In [36]:
for z,a in enumerate(res2):
    if a>0:
        print(f"{df.columns[z]} = {a}")
    # print(z)

In [37]:
res=df.isnull().sum() * 100 / df.shape[0]


In [38]:
res[:]

id                          0.000000
class                       0.000000
cap-diameter                0.000128
does-bruise-or-bleed        0.000000
stem-height                 0.000000
stem-width                  0.000000
has-ring                    0.000000
season_cos                  0.000000
season_sin                  0.000000
freq-cap-shape-p            0.000000
freq-cap-shape-e            0.000000
freq-cap-surface-p          0.000000
freq-cap-surface-e          0.000000
freq-cap-color-p            0.000000
freq-cap-color-e            0.000000
freq-gill-attachment-p      0.000000
freq-gill-attachment-e      0.000000
freq-gill-spacing-p         0.000000
freq-gill-spacing-e         0.000000
freq-gill-color-p           0.000000
freq-gill-color-e           0.000000
freq-stem-root-p            0.000000
freq-stem-root-e            0.000000
freq-stem-surface-p         0.000000
freq-stem-surface-e         0.000000
freq-stem-color-p           0.000000
freq-stem-color-e           0.000000
f

In [39]:
for z,a in enumerate(res):
    if a>0:
        print(f"{df.columns[z]} = {a}")
    # print(z)

cap-diameter = 0.0001283307854325309


In [40]:
x=df.dropna()
x=x.isnull().sum() * 100 / df.shape[0]


In [41]:
x

id                          0.0
class                       0.0
cap-diameter                0.0
does-bruise-or-bleed        0.0
stem-height                 0.0
stem-width                  0.0
has-ring                    0.0
season_cos                  0.0
season_sin                  0.0
freq-cap-shape-p            0.0
freq-cap-shape-e            0.0
freq-cap-surface-p          0.0
freq-cap-surface-e          0.0
freq-cap-color-p            0.0
freq-cap-color-e            0.0
freq-gill-attachment-p      0.0
freq-gill-attachment-e      0.0
freq-gill-spacing-p         0.0
freq-gill-spacing-e         0.0
freq-gill-color-p           0.0
freq-gill-color-e           0.0
freq-stem-root-p            0.0
freq-stem-root-e            0.0
freq-stem-surface-p         0.0
freq-stem-surface-e         0.0
freq-stem-color-p           0.0
freq-stem-color-e           0.0
freq-veil-type-p            0.0
freq-veil-type-e            0.0
freq-veil-color-p           0.0
freq-veil-color-e           0.0
freq-rin

In [42]:
df

,id,class,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,season_cos,season_sin,freq-cap-shape-p,...,freq-veil-type-p,freq-veil-type-e,freq-veil-color-p,freq-veil-color-e,freq-ring-type-p,freq-ring-type-e,freq-spore-print-color-p,freq-spore-print-color-e,freq-habitat-p,freq-habitat-e
0,0,-1,8.80,-1,4.51,15.39,-1,-1,0,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
1,1,1,4.51,-1,4.79,6.48,1,0,-1,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.066461,0.000310,0.0,0.0,0.675260,0.726853
2,2,-1,6.94,-1,6.85,9.93,-1,0,-1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.039805,0.073684
3,3,-1,3.88,-1,4.16,6.53,-1,0,1,0.203000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
4,4,-1,5.85,-1,3.37,8.36,-1,-1,0,0.429585,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.179948,0.104868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,-1,9.29,1,12.14,18.81,1,0,1,0.203000,...,0.061266,0.038887,0.082471,0.098066,0.015739,0.026103,0.0,0.0,0.675260,0.726853
3116941,3116941,-1,10.88,1,6.65,26.97,-1,0,1,0.117645,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
3116942,3116942,1,7.82,-1,9.51,11.06,1,-1,0,0.429585,...,0.000000,0.000000,0.082471,0.098066,0.066461,0.000310,0.0,0.0,0.675260,0.726853
3116943,3116943,-1,9.45,1,9.13,17.77,1,0,1,0.028815,...,0.000000,0.000000,0.000000,0.000000,0.020403,0.023295,0.0,0.0,0.675260,0.726853


In [43]:
df_test

,id,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,season_cos,season_sin,freq-cap-shape-p,freq-cap-shape-e,...,freq-veil-type-p,freq-veil-type-e,freq-veil-color-p,freq-veil-color-e,freq-ring-type-p,freq-ring-type-e,freq-spore-print-color-p,freq-spore-print-color-e,freq-habitat-p,freq-habitat-e
0,3116945,8.64,1,11.13,17.12,1,-1,0,0.429585,0.498328,...,0.061266,0.038887,0.082471,0.098066,0.015739,0.026103,0.0,0.0,0.675260,0.726853
1,3116946,6.90,-1,1.27,10.75,-1,-1,0,0.043065,0.025073,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
2,3116947,2.00,-1,6.18,3.14,-1,1,0,0.144439,0.051235,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
3,3116948,3.47,-1,4.98,8.51,1,0,1,0.429585,0.498328,...,0.000000,0.000000,0.017548,0.000079,0.066461,0.000310,0.0,0.0,0.675260,0.726853
4,3116949,6.17,-1,6.73,13.70,1,0,1,0.429585,0.498328,...,0.000000,0.000000,0.000093,0.021476,0.000000,0.000000,0.0,0.0,0.675260,0.726853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,-1,2.67,1.35,-1,0,1,0.429585,0.498328,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
2077960,5194905,3.12,-1,2.69,7.38,-1,-1,0,0.429585,0.498328,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.179948,0.104868
2077961,5194906,5.73,-1,6.16,9.74,1,-1,0,0.429585,0.498328,...,0.000000,0.000000,0.082471,0.098066,0.066461,0.000310,0.0,0.0,0.675260,0.726853
2077962,5194907,5.03,-1,6.00,3.46,-1,-1,0,0.144439,0.051235,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853


In [44]:
len(df.columns),len(df_test.columns)

(37, 36)

In [45]:
df['has-ring'].value_counts(dropna=False)


has-ring
-1    2368820
 1     747982
 0        143
Name: count, dtype: int64

In [46]:
for z,a in enumerate(x):
    if a>0:
        print(f"{df.columns[z]} = {a}")
    # print(z)

In [47]:
# x=df['cap-diameter'].median

In [48]:
X = df[df.columns[2:]]
y = df['class'].values

# Let's split the data into training and testing sets.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=13
)

In [49]:
np.unique(y)

array([-1,  1], dtype=int64)

In [ ]:
def model_classification_report(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"Classification report for the model {model_name}:\n",
        classification_report(y_test, y_pred),
    )

In [ ]:
model_HGB_default = HistGradientBoostingClassifier()

In [118]:
model_HGB_default.fit(X_train, y_train)

HistGradientBoostingClassifier()

In [119]:
y= model_HGB_default.predict(df_test[df_test.columns[1:]])


In [120]:
y

array([-1,  1,  1, ...,  1, -1, -1], dtype=int64)

In [121]:
predicts = ['p' if i ==1 else 'e'for i in y] 

In [122]:
predicts

['e',
 'p',
 'p',
 'p',
 'e',
 'e',
 'e',
 'p',
 'p',
 'e',
 'p',
 'p',
 'e',
 'e',
 'e',
 'p',
 'p',
 'p',
 'p',
 'e',
 'p',
 'e',
 'p',
 'e',
 'e',
 'p',
 'e',
 'p',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'p',
 'e',
 'e',
 'p',
 'e',
 'e',
 'p',
 'e',
 'p',
 'p',
 'p',
 'p',
 'p',
 'e',
 'p',
 'p',
 'p',
 'e',
 'e',
 'e',
 'p',
 'e',
 'p',
 'p',
 'e',
 'e',
 'e',
 'p',
 'p',
 'p',
 'e',
 'p',
 'e',
 'e',
 'e',
 'p',
 'e',
 'e',
 'p',
 'p',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'p',
 'e',
 'e',
 'e',
 'p',
 'e',
 'p',
 'e',
 'p',
 'e',
 'p',
 'p',
 'p',
 'e',
 'p',
 'e',
 'p',
 'p',
 'p',
 'p',
 'p',
 'e',
 'e',
 'p',
 'e',
 'e',
 'p',
 'e',
 'e',
 'e',
 'e',
 'p',
 'e',
 'e',
 'e',
 'p',
 'p',
 'e',
 'p',
 'p',
 'e',
 'e',
 'e',
 'e',
 'e',
 'p',
 'p',
 'p',
 'p',
 'e',
 'p',
 'e',
 'p',
 'p',
 'e',
 'p',
 'p',
 'e',
 'e',
 'p',
 'p',
 'p',
 'p',
 'e',
 'p',
 'e',
 'p',
 'e',
 'e',
 'e',
 'e',
 'e',
 'p',
 'p',
 'e',
 'e',
 'p',
 'p',
 'p',
 'e',
 'e',
 'p',
 'e',
 'p',
 'p'

In [129]:
print(len(predicts), len(df_sample_submission))  # بررسی طول‌ها


2077964 2077964


In [128]:
print(np.isnan(predicts).sum())  # بررسی NaN
print(np.isinf(predicts).sum())  # بررسی Inf


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [125]:
print(type(predicts))  # بررسی نوع predicts


<class 'list'>


In [124]:
df_sample_submission['class'] = predicts

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
df_sample_submission

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


# another way lets test it and see how does it work 

In [51]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Activation, Dropout
from tensorflow.keras.regularizers import l2

In [58]:
X = df[df.columns[2:]]
y = df['class'].values

# Let's split the data into training and testing sets.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=13
)

In [59]:
X


,cap-diameter,does-bruise-or-bleed,stem-height,stem-width,has-ring,season_cos,season_sin,freq-cap-shape-p,freq-cap-shape-e,freq-cap-surface-p,...,freq-veil-type-p,freq-veil-type-e,freq-veil-color-p,freq-veil-color-e,freq-ring-type-p,freq-ring-type-e,freq-spore-print-color-p,freq-spore-print-color-e,freq-habitat-p,freq-habitat-e
0,8.80,-1,4.51,15.39,-1,-1,0,0.203000,0.233815,0.083374,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
1,4.51,-1,4.79,6.48,1,0,-1,0.429585,0.498328,0.078418,...,0.000000,0.000000,0.000000,0.000000,0.066461,0.000310,0.0,0.0,0.675260,0.726853
2,6.94,-1,6.85,9.93,-1,0,-1,0.203000,0.233815,0.083374,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.039805,0.073684
3,3.88,-1,4.16,6.53,-1,0,1,0.203000,0.233815,0.088806,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
4,5.85,-1,3.37,8.36,-1,-1,0,0.429585,0.498328,0.011279,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.179948,0.104868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,9.29,1,12.14,18.81,1,0,1,0.203000,0.233815,0.000000,...,0.061266,0.038887,0.082471,0.098066,0.015739,0.026103,0.0,0.0,0.675260,0.726853
3116941,10.88,1,6.65,26.97,-1,0,1,0.117645,0.116549,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.794164,0.795442,0.0,0.0,0.675260,0.726853
3116942,7.82,-1,9.51,11.06,1,-1,0,0.429585,0.498328,0.033584,...,0.000000,0.000000,0.082471,0.098066,0.066461,0.000310,0.0,0.0,0.675260,0.726853
3116943,9.45,1,9.13,17.77,1,0,1,0.028815,0.040966,0.054672,...,0.000000,0.000000,0.000000,0.000000,0.020403,0.023295,0.0,0.0,0.675260,0.726853


In [60]:
y

array([-1,  1, -1, ...,  1, -1,  1], dtype=int64)

In [55]:
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train)
# X_cv_mapped_scaled = scaler.transform(x_cv)
X_test_mapped_scaled = scaler.transform(X_test)
# X_test_mapped_scaled
X_train_mapped_scaled

array([[ 0.88644697, -0.46110993,  0.36019363, ..., -0.27603119,
         0.64996099,  0.65587049],
       [-1.13229407, -0.46110993, -1.15109378, ..., -0.27603119,
        -1.73648874, -1.60845377],
       [-0.69156786, -0.46110993, -0.45471624, ..., -0.27603119,
         0.64996099,  0.65587049],
       ...,
       [-0.40778317, -0.46110993,  1.65664117, ...,  3.11994157,
        -1.23326006, -1.42322183],
       [-0.67651867, -0.46110993, -0.88069186, ...,  3.28140431,
        -1.23326006, -1.42322183],
       [-0.57547412, -0.46110993, -2.35123378, ..., -0.26479584,
         0.64996099,  0.65587049]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

# نرمالیزه کردن داده‌ها
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(df_test[df_test.columns[1:]])

# ساخت مدل RandomForestClassifier
model_RF = RandomForestClassifier()

# آموزش مدل
model_RF.fit(X_train_scaled, y_train)

# پیش‌بینی مقادیر
df_sample_submission = model_RF.predict(X_test_scaled)

# نمایش پیش‌بینی‌ها
print(df_sample_submission)


In [57]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [61]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [62]:


nn_cv_mses = []

# For simplicity, I'm assuming some values for x_train, x_cv, x_test, y_train, y_cv
# Loop over 10 times. Each adding one more degree of polynomial higher than the last.
# for degree in range(1, 3):
#     # Add polynomial features
#     poly = PolynomialFeatures(degree, include_bias=False)
#     X_train_mapped = poly.fit_transform(x_train)
#     X_cv_mapped = poly.transform(x_cv)
#     X_test_mapped = poly.transform(x_test)
    
    # Scale the features using the z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train)
# X_cv_mapped_scaled = scaler.transform(x_cv)
X_test_mapped_scaled = scaler.transform(X_test)

# Start model here
model = Sequential([
    Input(shape=(X_train_mapped_scaled.shape[1],)),  
    Dense(128, kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.1),
    
    Dense(128, kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.1),
    
    Dense(64, kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.1),
    # Modify the output layer to have one unit with sigmoid activation
    Dense(1, activation='tanh')
])
# Compile the model with binary cross-entropy loss
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy()])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()
# Train the model
# model.fit(X_train_mapped_scaled, y_train, epochs=100, verbose=0)

# # Compute the training accuracy
# yhat_train = model.predict(X_train_mapped_scaled)
# train_accuracy = binary_accuracy(y_train, yhat_train)

# # Compute the cross-validation accuracy
# yhat_cv = model.predict(X_cv_mapped_scaled)
# cv_accuracy = binary_accuracy(y_cv, yhat_cv)
# nn_cv_accuracies.append(cv_accuracy)
# print(f"finished degree number{degree} and cv = {nn_cv_accuracies[degree]} ")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,721 (120.00 KB)

 Trainable params: 30,081 (117.50 KB)

 Non-trainable params: 640 (2.50 KB)

In [96]:


nn_cv_mses = []

# For simplicity, I'm assuming some values for x_train, x_cv, x_test, y_train, y_cv
# Loop over 10 times. Each adding one more degree of polynomial higher than the last.
# for degree in range(1, 3):
#     # Add polynomial features
#     poly = PolynomialFeatures(degree, include_bias=False)
#     X_train_mapped = poly.fit_transform(x_train)
#     X_cv_mapped = poly.transform(x_cv)
#     X_test_mapped = poly.transform(x_test)
    
    # Scale the features using the z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train)
# X_cv_mapped_scaled = scaler.transform(x_cv)
X_test_mapped_scaled = scaler.transform(X_test)

# Start model here
model = Sequential([
    Dense(64, activation='tanh', input_shape=(X_train_mapped_scaled.shape[1],)),
    Dense(1, activation='tanh')
])
# from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error', metrics=['accuracy'])

model.summary()
# Train the model
# model.fit(X_train_mapped_scaled, y_train, epochs=100, verbose=0)

# # Compute the training accuracy"
# yhat_train = model.predict(X_train_mapped_scaled)
# train_accuracy = binary_accuracy(y_train, yhat_train)

# # Compute the cross-validation accuracy
# yhat_cv = model.predict(X_cv_mapped_scaled)
# cv_accuracy = binary_accuracy(y_cv, yhat_cv)
# nn_cv_accuracies.append(cv_accuracy)
# print(f"finished degree number{degree} and cv = {nn_cv_accuracies[degree]} ")


c:\Users\Ehsan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,369 (9.25 KB)

 Trainable params: 2,369 (9.25 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
len(X.columns)

35

In [64]:
print(np.isinf(X_train_mapped_scaled).sum())  # تعداد مقادیر Inf
print(np.isinf(X_train_mapped_scaled).sum())  # بررسی مقادیر Infinity


0
0


In [ ]:
# X_train_mapped_scaled = np.nan_to_num(X_train_mapped_scaled, nan=0.0, posinf=0.0, neginf=0.0)


In [66]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train)
X_test_mapped_scaled = scaler.transform(X_test)


In [ ]:
X_train_mapped_scaled = np.nan_to_num(X_train_mapped_scaled, nan=0.0, posinf=0.0, neginf=0.0)


In [68]:
history = model.fit(X_train_mapped_scaled, y_train, epochs=10, verbose=1)
print(history.history['loss'])


Epoch 1/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 111s 1ms/step - accuracy: 0.5151 - loss: 0.1501
Epoch 2/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 101s 1ms/step - accuracy: 0.5319 - loss: 0.1034
Epoch 3/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 97s 1ms/step - accuracy: 0.5328 - loss: 0.1006
Epoch 4/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 98s 1ms/step - accuracy: 0.5329 - loss: 0.0991
Epoch 5/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 96s 1ms/step - accuracy: 0.5335 - loss: 0.0986
Epoch 6/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 98s 1ms/step - accuracy: 0.5319 - loss: 0.0972
Epoch 7/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 97s 1ms/step - accuracy: 0.5330 - loss: 0.0961
Epoch 8/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 102s 1ms/step - accuracy: 0.5329 - loss: 0.0963
Epoch 9/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 98s 1ms/step - accuracy: 0.5329 - loss: 0.0956
Epoch 10/10
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 93s 1ms/step - accuracy: 0.5327 - loss: 0.0945
[0.11859017610549927, 0.10250284522771835, 0.10008145123720169, 0.09877582639455795, 0

In [87]:
import numpy as np
print(np.isnan(X_train_mapped_scaled).sum())
print(np.isnan(y_train).sum())


2
0


In [54]:
# model.fit(X_train_mapped_scaled, y_train, epochs=1, verbose=0)
# yhat_train = model.predict(X_train_mapped_scaled)
# train_mse = mean_squared_error(y_train, yhat_train) / 2

# # Compute the cross-validation MSE
# yhat_cv = model.predict(X_cv_mapped_scaled)
# cv_mse = mean_squared_error(y_cv, yhat_cv) / 2
# nn_cv_mses.append(cv_mse)

# print(f"Finished training with CV MSE = {cv_mse:.4f}")
len(df_test[df_test.columns[1:]].columns)

35

In [69]:
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(df_test[df_test.columns[1:]])
df_sample_submission = model.predict(X_train_mapped_scaled)


64937/64937 ━━━━━━━━━━━━━━━━━━━━ 46s 714us/step


In [70]:
# X_train_mapped_scaled
print(np.isnan(X_train_mapped_scaled).sum())


0


# final result

In [71]:
df_sample_submission

array([[-0.9959129],
       [ 1.       ],
       [ 1.       ],
       ...,
       [ 1.       ],
       [ 1.       ],
       [-0.9999998]], dtype=float32)

In [72]:
predicts = ['p' if i >0 else 'e'for i in df_sample_submission] 


In [73]:
x=pd.DataFrame(predicts)

In [74]:
x

,0
0,e
1,p
2,p
3,p
4,p
...,...
2077959,p
2077960,p
2077961,p
2077962,p


In [77]:
x[0]

0          e
1          p
2          p
3          p
4          p
          ..
2077959    p
2077960    p
2077961    p
2077962    p
2077963    e
Name: 0, Length: 2077964, dtype: object

In [80]:
df_sample_submission['class']


0          e
1          e
2          e
3          e
4          e
          ..
2077959    e
2077960    e
2077961    e
2077962    e
2077963    e
Name: class, Length: 2077964, dtype: object

In [81]:
df_sample_submission['class'] = x[0]


In [82]:
x.to_csv('final2.csv',index=False)


In [75]:
print(np.isnan(X_train_mapped_scaled).sum())
print(np.isnan(X_test_mapped_scaled).sum())


0
2


In [83]:
df_sample_submission.to_csv('final4.csv',index=False)

In [145]:
df_sample_submission

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [72]:
import numpy as np

# چک کردن و پاکسازی داده‌ها
df_test_cleaned = df_test.dropna()
df_test_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test_cleaned.dropna(inplace=True)

# اسکیلینگ داده‌ها
scaler = StandardScaler()
X_test_mapped_scaled = scaler.fit_transform(df_test_cleaned[df_test_cleaned.columns[1:]])

# چک کردن مقادیر گمشده در اسکیلینگ
if np.isnan(X_test_mapped_scaled).any():
    print("X_test_mapped_scaled contains NaN values.")

# پیش‌بینی مقادیر با مدل عصبی
df_sample_submission = model.predict(X_test_mapped_scaled)

# چک کردن مقادیر گمشده در پیش‌بینی
if np.isnan(df_sample_submission).any():
    print("Model output contains NaN values.")


64937/64937 ━━━━━━━━━━━━━━━━━━━━ 41s 631us/step
Model output contains NaN values.


In [76]:
print(X_train.std(axis=0))  # انحراف معیار ستون‌ها


cap-diameter                4.651415
does-bruise-or-bleed        0.760474
stem-height                 2.699686
stem-width                  8.089517
has-ring                    0.854248
season_cos                  0.582021
season_sin                  0.616856
freq-cap-shape-p            0.150700
freq-cap-shape-e            0.191500
freq-cap-surface-p          0.053648
freq-cap-surface-e          0.058984
freq-cap-color-p            0.155347
freq-cap-color-e            0.209386
freq-gill-attachment-p      0.088799
freq-gill-attachment-e      0.062066
freq-gill-spacing-p         0.211142
freq-gill-spacing-e         0.186616
freq-gill-color-p           0.080293
freq-gill-color-e           0.135423
freq-stem-root-p            0.012048
freq-stem-root-e            0.017510
freq-stem-surface-p         0.038433
freq-stem-surface-e         0.044582
freq-stem-color-p           0.118734
freq-stem-color-e           0.182328
freq-veil-type-p            0.013506
freq-veil-type-e            0.008572
f

In [141]:
x.shape,df_sample_submission.shape

((2077964, 1), (2077964, 2))